Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer

In [7]:
from nltk.stem import WordNetLemmatizer

In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [11]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [18]:
tweets = test_df['text'].tolist()

**Analizador de sentimentos pré-treinado do NLTK (VADER)**

In [22]:
sia = SentimentIntensityAnalyzer()

In [9]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(train_df[train_df["target"] == 1]["text"].values[1])

{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.34}

In [10]:
sia.polarity_scores(train_df[train_df["target"] == 0]["text"].values[1])

{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

In [24]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

In [54]:
sentimento = []
correct = 0
num = 0

In [55]:
for tweet in tweets:
    sentimento.append(is_positive(tweet))
    if is_positive(tweet) == True and train_df['target'][num] == 1:
        correct+=1
    elif is_positive(tweet) == False and train_df['target'][num] == 0:
        correct+=1
    else:
        pass
    num +=1

In [61]:
print(correct*100 / len(tweets))

54.121973643885994


In [45]:
import operator

In [43]:
test_df['sentimento'] = list(map(operator.not_, sentimento))

In [62]:
test_df_sentimento = test_df

In [63]:
test_df_sentimento.head()

,id,keyword,location,text,sentimento
0,0,NaN,NaN,Just happened a terrible car crash,True
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",False
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",True
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,True
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,True


**Personalizando a análise de sentimento do NLTK**

In [65]:
unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])

def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

#Fazer uma lista das palavras positivas e negativas!!

In [64]:
def extract_features(text):
    features = dict()
    wordcount = 0
    compound_scores = list()
    positive_scores = list()
    for sentence in tweets:
        for word in nltk.word_tokenize(sentence):
            if word.lower() in top_100_positive:
                wordcount += 1
        compound_scores.append(sia.polarity_scores(sentence)["compound"])
        positive_scores.append(sia.polarity_scores(sentence)["pos"])

In [ ]:
for sentence in tweets:
    for word in nltk.word_tokenize(sentence):
        if word.lower() in top_100_positive:
            wordcount += 1
    compound_scores.append(sia.polarity_scores(sentence)["compound"])
    positive_scores.append(sia.polarity_scores(sentence)["pos"])